# Setup

In [2]:
# just run the first time notebook is used. Python version at least 3.8 (did not install successfully with 3.7)
#pip install malariagen_data==7.1.3

  Using cached malariagen_data-7.1.3-py3-none-any.whl (102 kB)
  Using cached bokeh-3.0.3-py3-none-any.whl (16.5 MB)
  Using cached ipyleaflet-0.17.2-py3-none-any.whl (3.7 MB)
  Using cached ipinfo-4.4.2-py3-none-any.whl (23 kB)
  Using cached jupyter_dash-0.4.2-py3-none-any.whl (23 kB)
  Using cached scikit_allel-1.3.5-cp38-cp38-manylinux2010_x86_64.whl (7.2 MB)
  Using cached xarray-2022.12.0-py3-none-any.whl (969 kB)
  Using cached dash_cytoscape-0.3.0-py3-none-any.whl (3.6 MB)
  Using cached dask-2022.12.1-py3-none-any.whl (1.1 MB)
  Using cached pandas-1.5.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)
  Using cached statsmodels-0.13.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.9 MB)
  Using cached zarr-2.13.3-py3-none-any.whl (191 kB)
  Using cached scipy-1.10.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
  Using cached plotly-5.11.0-py2.py3-none-any.whl (15.3 MB)
  Using cached fsspec-2022.11.0-py3-none-any.whl (139 k

  Using cached frozenlist-1.3.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (161 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached yarl-1.8.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (262 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
  Using cached locket-1.0.0-py2.py3-none-any.whl (4.4 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached google_resumable_media-2.4.0-py2.py3-none-any.whl (77 kB)
  Using cached google_cloud_core-2.3.2-py2.py3-none-any.whl (29 kB)
  Using cached google_api_core-2.11.0-py3-none-any.whl (120 kB)
  Using cached googleapis_common_protos-1.57.1-py2.py3-none-any.whl (218 kB)
  Using cached google_crc32c-1.5.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (32 kB)


  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 6.0.0
    Uninstalling importlib-metadata-6.0.0:
      Successfully uninstalled importlib-metadata-6.0.0
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import relevant packages
from pyprojroot import here
import pandas as pd
import gcsfs
import zarr
from dask_gateway import Gateway
import dask
import dask.array as da
import numpy as np
from fsspec.implementations.zip import ZipFileSystem
import os
import allel
import malariagen_data

In [7]:
#define the chromosomes and crosses to be investigated
chroms=["2L","2R","3L","3R","X"]
crosses=["B1", "B3", "B5", "K2", "K4", "K6"]

In [8]:
for chrom in chroms:
    for cross in crosses:
        cross=cross
        contig=chrom

In [10]:
# Sets up the data
ag3 = malariagen_data.Ag3("gs://vo_agam_release/")

In [11]:
# Sets up paths to gcs buckets
gcs_source_zarr_path_template = 'vo_agam_release/v3/snp_genotypes/per_sample/AG1000G-X/zarr/{sample}.zarr.zip'
gcs_output_dir_path_template = 'vo_agam_release/v3/snp_genotypes/per_sample/AG1000G-X/zarr/potential_fathers_combined.zarr'
gcs_final_output_zarr_path = 'vo_agam_release/v3/snp_genotypes/per_sample/AG1000G-X/zarr/potential_fathers_filtered.zarr'

In [12]:
# Sets up access to gcs - project might need to be modified
gcs = gcsfs.GCSFileSystem(
    project='malariagen-jupyterhub', 
    token='anon',
    block_size=2**18,  # minimum, reduces memory footprint when reading from zip files
)

In [13]:
# Sets up access to sample lists - depends on local file system
df_crosses = pd.read_csv(here() / "metadata_original_samples" / "original_samples.tsv", sep="\t")

In [17]:
# Defines sample lists for the cross
f2_partner_sample_id = [psi[:2] for psi in df_crosses.partner_sample_id]
offsprings = [f2_psi == cross for f2_psi in f2_partner_sample_id]
samples = list(df_crosses[offsprings].original_sample_id + '-C')
print(samples)

['AC0335-C', 'AC0336-C', 'AC0337-C', 'AC0338-C', 'AC0339-C', 'AC0340-C', 'AC0341-C', 'AC0342-C', 'AC0343-C', 'AC0344-C', 'AC0345-C', 'AC0346-C', 'AC0347-C', 'AC0348-C', 'AC0349-C']


# Function definitions

In [22]:
# Functions dealing with opening input zarr files
def open_input(samples, seq_id, field):
    
    if isinstance(samples, list):
        zl = []
        for s in samples:
            zl.append(_open_input(s, seq_id, field))
        return da.concatenate(zl, axis=1)
    elif isinstance(samples, str):
        return _open_input(samples, seq_id, field)
    else:
        raise TypeError(type(samples))
        
        
def _open_input(sample, seq_id, field):
    gcs_path = gcs_source_zarr_path_template.format(sample=sample)
    array_path = f'{sample}/{seq_id}/{field}'

    # Get the zarr store from the zip file
    zip_file = gcs.open(gcs_path)
    zip_fs = ZipFileSystem(zip_file) 
    zarr_store = zarr.storage.KVStore(zip_fs.get_mapper("/"))

    # open and load the array
    root = zarr.open(store=zarr_store, mode='r')
    z = root[array_path][:]
    
    return z

In [23]:
# Checks that the input us correctly accessed
def check_array_setup(samples, seq_id, field):
    
    # Returns information about data array for the first sample for the specified 
    # sample set, seq_id and field.

    z = open_input(samples=samples[0], seq_id=seq_id, field=field)
    return z.dtype, z.shape

In [24]:
# Utility function
def my_not(a):
    return not a

not_arr = np.vectorize(my_not)

# Main code

In [76]:
#choose a data set to try out the code
cross="B3"
contig="2L"

In [77]:
# Checks that the data is correctly accessed

check_array_setup(samples=samples, seq_id = contig, field = 'calldata/GT')

(dtype('int8'), (48525747, 1, 2))

In [78]:
# Accesses the data
def access_data(samples=samples, contig=contig, field='calldata/GT'):
    z = open_input(samples=samples, seq_id = contig, field = 'calldata/GT')
    return z

In [79]:
z=access_data(samples=samples, contig=contig, field='calldata/GT')

In [81]:
# Casts the data into a Genotype Array
def make_genotype_array(z=z):
    gt = allel.GenotypeDaskArray(z)
    return gt

In [82]:
gt=make_genotype_array(z=z)

In [84]:
# Rechunks the data to make computation more efficient
def rechunk_data(gt=gt):
    gt_r = gt.rechunk((66580, 9, 2))
    gt_r_loc = gt_r.compute()
    return gt_r, gt_r_loc

In [85]:
gt_r, gt_r_loc=rechunk_data(gt=gt)

In [88]:
# Computes allele counts
def compute_allele_counts(gt_r_loc=gt_r_loc):
    ac_loc = gt_r_loc.count_alleles()
    return ac_loc

In [89]:
ac_loc=compute_allele_counts(gt_r_loc=gt_r_loc)

In [91]:
# Identifies triallelic sites
def identify_triallelic_sites(ac_loc=ac_loc):
    is_triallel = (ac_loc.allelism() > 2)
    return is_triallel

In [92]:
is_triallel=identify_triallelic_sites(ac_loc=ac_loc)

In [94]:
# Sets up the result array and identifies heterozygous, homozygous and missing sites/samples
def results_array(gt_r=gt_r, gt_r_loc=gt_r_loc):
    res = np.zeros((gt_r_loc.shape[0], 2), dtype='i4')
    gt_het = gt_r_loc.is_het()
    gt_hom = gt_r_loc.is_hom()
    gt_miss = gt_r_loc.is_missing()
    return res, gt_het, gt_hom, gt_miss

In [95]:
res, gt_het, gt_hom, gt_miss=results_array(gt_r=gt_r, gt_r_loc=gt_r_loc)

In [101]:
# Identifies sites where some samples are homozygous and sites where most samples are missing
def mixed_sites(gt_hom=gt_hom, gt_miss=gt_miss):
    is_some_hom = (da.sum(gt_hom, axis = 1).compute() >= 1)
    is_most_miss = (da.sum(gt_miss, axis = 1).compute() >= 5)
    return is_some_hom, is_most_miss

In [102]:
is_some_hom, is_most_miss=mixed_sites(gt_hom=gt_hom, gt_miss=gt_miss)

In [105]:
# Computes actual data
def compute_actual_data(gt_r_loc=gt_r_loc, \
                        ac_loc=ac_loc, \
                        is_most_miss=is_most_miss, \
                        is_triallel=is_triallel, \
                        is_some_hom=is_some_hom, \
                        gt_miss=gt_miss, \
                        gt_hom=gt_hom, \
                        gt_het=gt_het):
    for i in range(gt_r_loc.shape[0]):
        if is_most_miss[i]: # If mostly missing, return missing
            res[i] = np.array([-1,-1])
        elif is_triallel[i]: # if triallelic, return homozygous for most frequent allel
            maj_val = np.argmax(ac_loc[i])
            res[i] = np.array([maj_val,maj_val])
        elif is_some_hom[i]: # If some are homozygous, return the homozygous allel
            res[i] = gt_r_loc[i][gt_hom[i]][0]
        else: #Otherwise, return the heterozygous
            not_miss = not_arr(gt_miss[i])
            res[i] = gt_r_loc[i][not_miss][0]
    return res

In [106]:
res = compute_actual_data(gt_r_loc=gt_r_loc, \
                        ac_loc=ac_loc, \
                        is_most_miss=is_most_miss, \
                        is_triallel=is_triallel, \
                        is_some_hom=is_some_hom, \
                        gt_miss=gt_miss, \
                        gt_hom=gt_hom, \
                        gt_het=gt_het)

In [108]:
# Accesses data on sites
def access_site_data(contig=contig):
    pos = ag3.snp_sites(contig,'POS').compute()
    ref = ag3.snp_sites(contig,'REF').compute()
    alt = ag3.snp_sites(contig,'ALT').compute()
    return pos, ref, alt

In [109]:
pos, ref, alt =access_site_data(contig=contig)

In [113]:
# Open the template (the example header file needs to be in the same directory as this worksheet)
def open_template():
    template = open("AC0300-C_header.txt", "r")

    lines = template.read().split('\n')

    template.close()
    return lines

In [114]:
lines = open_template()

In [115]:
# Updates the template for this specific run
def update_template(lines=lines, contig=contig, cross=cross):
        
    vcom = lines[-3].split(' ')
    head = lines[-2].split('\t')

    vcom[-1] = cross + "-" + contig + ".vcf.gz"
    head[-1] = cross + "-" + contig 

    lines[-3] = " ".join(vcom)
    lines[-2] = "\t".join(head)
    return lines

In [116]:
lines = update_template(lines=lines, contig=contig, cross=cross)

In [118]:
# Defines the output file name 
def output_file_name(prefix, cross=cross, contig=contig):
    file_name = prefix + cross + "-" + contig + ".vcf"
    return file_name

In [147]:
file_name=output_file_name(prefix="test", cross=cross, contig=contig)

In [148]:
file_name

'testB3-2L.vcf'

In [176]:
# Writes header and data to file
def write_vcf(file_name=file_name, lines=lines, pos=pos, contig=contig, res=res, ref=ref, alt=alt):
        res_file = open(file_name, "w")
        res_file.write("\n".join([line for line in lines[:-1]]))
        for i in range(0,len(pos)):#100):
            res_file.write("\n" + contig + "\t" + str(pos[i]) + "\t.\t" + ref[i].decode("utf-8") \
                           + "\t" + alt[i,0].decode("utf-8") + "," + alt[i,1].decode("utf-8") + "," \
                           + alt[i,2].decode("utf-8") + "\t.\t.\t.\tGT:AD:DP:GQ:PL:SB\t" + str(res[i,0]) \
                           + "/" + str(res[i,1]) + ":.:.:.:.:.")

In [177]:
write_vcf(file_name=file_name, lines=lines, pos=pos, contig=contig, res=res, ref=ref, alt=alt)

In [141]:
# Closes the file
def close_file(res_file=res_file):
    res_file.close()

In [142]:
close_file(res_file=res_file)

In [143]:
def file_check(file_name=file_name):
    file_check = open(file_name, "r")

    file_check_lines = file_check.read(10_000).split('\n')

    file_check.close()
    return file_check_lines

# Put the functions together

In [ ]:
#loop through the chromosomes and crosses to be investigated, writing a vcf file for each. VCF files written in same
#directory as this worksheet.

In [180]:
for chrom in chroms:
    for cross in crosses:
        cross=cross
        print(cross)
        contig=chrom
        seqid=contig
        print(chrom)
        f2_partner_sample_id = [psi[:2] for psi in df_crosses.partner_sample_id]
        offsprings = [f2_psi == cross for f2_psi in f2_partner_sample_id]
        samples = list(df_crosses[offsprings].original_sample_id + '-C')
        print(samples)
        z= access_data(samples=samples, contig=contig, field='calldata/GT')
        print("data accessed for"+cross+"and"+chrom)
        gt= make_genotype_array(z=z)
        print("genotype_array_made")
        print(gt[:5])
        gt_r, gt_r_loc=rechunk_data(gt=gt)
        print("data_rechunked")
        print(gt_r[:5])
        print(gt_r_loc[:5])
        ac_loc=compute_allele_counts(gt_r_loc=gt_r_loc)
        print("allele counts computed")
        print(ac_loc[:5])
        is_triallel=identify_triallelic_sites(ac_loc=ac_loc)
        print("triallelic_sites_identified")
        print(is_triallel[:5])
        res, gt_het, gt_hom, gt_miss = results_array(gt_r=gt_r, gt_r_loc=gt_r_loc)
        print("made results array")
        print(res[:5])
        print(gt_het[:5])
        print(gt_hom[:5])
        print(gt_miss[:5])
        is_some_hom, is_most_miss = mixed_sites(gt_hom=gt_hom, gt_miss=gt_miss)
        print("calculated mixed sites")
        print(is_some_hom[:5])
        print(is_most_miss[:5])
        res = compute_actual_data(gt_r_loc=gt_r_loc, \
                        ac_loc=ac_loc, \
                        is_most_miss=is_most_miss, \
                        is_triallel=is_triallel, \
                        is_some_hom=is_some_hom, \
                        gt_miss=gt_miss, \
                        gt_hom=gt_hom, \
                        gt_het=gt_het)
        print("actual data computed")
        print(res[:5])
        pos, ref, alt = access_site_data(contig=contig)
        print("site data accessed")
        print(pos[:5])
        print(ref[:5])
        print(alt[:5])
        lines = open_template()
        lines = update_template(lines=lines, contig=contig, cross=cross)
        print("template updated")
        print(lines)
        file_name = output_file_name(prefix="cross", cross=cross, contig=contig)
        print("file name is" + file_name)
        write_vcf(file_name=file_name, lines=lines, pos=pos, contig=contig, res=res, ref=ref, alt=alt)
        print("vcf file written")
        close_file()
        print("file closed")
        file_check()
        
        

B1
2L
['AC0352-C', 'AC0353-C', 'AC0354-C', 'AC0355-C', 'AC0356-C', 'AC0357-C', 'AC0358-C', 'AC0359-C', 'AC0360-C']
data accessed forB1and2L
genotype_array_made
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.

data_rechunked
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.

./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ./. ./. ./. ./.

allele counts computed
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0
0 0 0 0

triallelic_sites_identified
[False False False False False]
made results array
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]
[[False False False False False False False False False]
 [False False False False False False False Fa

vcf file written
file closed
B3
2L
['AC0362-C', 'AC0363-C', 'AC0364-C', 'AC0365-C', 'AC0366-C', 'AC0367-C', 'AC0368-C', 'AC0369-C', 'AC0370-C', 'AC0371-C', 'AC0372-C', 'AC0373-C', 'AC0374-C', 'AC0375-C', 'AC0376-C', 'AC0377-C', 'AC0378-C', 'AC0379-C', 'AC0380-C', 'AC0381-C']
data accessed forB3and2L
genotype_array_made
./. ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
./. ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
0/0 ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
0/0 ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
0/0 ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.

data_rechunked
./. ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
./. ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
0/0 ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
0/0 ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
0/0 ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.

./. ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
./. ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
0/0 ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
0/0 ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.
0/0 ./. ./. ./. ./. ... 0/0 ./. ./. ./. ./.

a

vcf file written
file closed
B5
2L
['AC0383-C', 'AC0384-C', 'AC0385-C', 'AC0386-C', 'AC0387-C', 'AC0388-C', 'AC0389-C', 'AC0390-C', 'AC0391-C', 'AC0392-C', 'AC0393-C']
data accessed forB5and2L
genotype_array_made
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.

data_rechunked
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.

./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. ./. ./. ./. ./.

allele counts computed
0 0 0 0
2 0 0 0
2 0 0 0
2 0 0 0
2 0 0 0

triallelic_sites_identified
[False False Fals

vcf file written
file closed
K2
2L
['AC0301-C', 'AC0302-C', 'AC0303-C', 'AC0304-C', 'AC0305-C', 'AC0306-C', 'AC0307-C', 'AC0308-C', 'AC0309-C', 'AC0310-C', 'AC0311-C', 'AC0312-C', 'AC0313-C', 'AC0314-C', 'AC0315-C', 'AC0316-C']
data accessed forK2and2L
genotype_array_made
./. ./. ./. ./. ./. ... ./. ./. ./. ./. 0/0
./. ./. ./. ./. ./. ... ./. ./. ./. ./. 0/0
./. ./. ./. ./. ./. ... 0/0 ./. ./. ./. 0/0
./. ./. ./. 0/0 ./. ... 0/0 ./. ./. ./. 0/0
./. ./. ./. 0/0 ./. ... 0/0 ./. ./. ./. 0/0

data_rechunked
./. ./. ./. ./. ./. ... ./. ./. ./. ./. 0/0
./. ./. ./. ./. ./. ... ./. ./. ./. ./. 0/0
./. ./. ./. ./. ./. ... 0/0 ./. ./. ./. 0/0
./. ./. ./. 0/0 ./. ... 0/0 ./. ./. ./. 0/0
./. ./. ./. 0/0 ./. ... 0/0 ./. ./. ./. 0/0

./. ./. ./. ./. ./. ... ./. ./. ./. ./. 0/0
./. ./. ./. ./. ./. ... ./. ./. ./. ./. 0/0
./. ./. ./. ./. ./. ... 0/0 ./. ./. ./. 0/0
./. ./. ./. 0/0 ./. ... 0/0 ./. ./. ./. 0/0
./. ./. ./. 0/0 ./. ... 0/0 ./. ./. ./. 0/0

allele counts computed
2 0 0 0
2 0 0 0
6 0 0 0
8 

vcf file written
file closed
K4
2L
['AC0318-C', 'AC0319-C', 'AC0320-C', 'AC0321-C', 'AC0322-C', 'AC0323-C', 'AC0324-C', 'AC0325-C', 'AC0326-C', 'AC0327-C', 'AC0328-C', 'AC0329-C', 'AC0330-C', 'AC0331-C', 'AC0332-C', 'AC0333-C']
data accessed forK4and2L
genotype_array_made
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. 0/0 ./. ./. ./.

data_rechunked
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. 0/0 ./. ./. ./.

./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. 0/0 ... ./. 0/0 ./. ./. ./.

allele counts computed
2 0 0 0
2 0 0 0
2 0 0 0
2 

...

vcf file written
file closed
B5
X
['AC0383-C', 'AC0384-C', 'AC0385-C', 'AC0386-C', 'AC0387-C', 'AC0388-C', 'AC0389-C', 'AC0390-C', 'AC0391-C', 'AC0392-C', 'AC0393-C']
data accessed forB5andX
genotype_array_made
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

data_rechunked
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

allele counts computed
22  0  0  0
22  0  0  0
22  0  0  0
22  0  0  0
22  0  0  0

triallelic_sites_identified

vcf file written
file closed
K2
X
['AC0301-C', 'AC0302-C', 'AC0303-C', 'AC0304-C', 'AC0305-C', 'AC0306-C', 'AC0307-C', 'AC0308-C', 'AC0309-C', 'AC0310-C', 'AC0311-C', 'AC0312-C', 'AC0313-C', 'AC0314-C', 'AC0315-C', 'AC0316-C']
data accessed forK2andX
genotype_array_made
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

data_rechunked
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

allele counts computed
32  0  0  0
32  0  0  0
32  

vcf file written
file closed
K4
X
['AC0318-C', 'AC0319-C', 'AC0320-C', 'AC0321-C', 'AC0322-C', 'AC0323-C', 'AC0324-C', 'AC0325-C', 'AC0326-C', 'AC0327-C', 'AC0328-C', 'AC0329-C', 'AC0330-C', 'AC0331-C', 'AC0332-C', 'AC0333-C']
data accessed forK4andX
genotype_array_made
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

data_rechunked
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

allele counts computed
32  0  0  0
32  0  0  0
32  

vcf file written
file closed
K6
X
['AC0335-C', 'AC0336-C', 'AC0337-C', 'AC0338-C', 'AC0339-C', 'AC0340-C', 'AC0341-C', 'AC0342-C', 'AC0343-C', 'AC0344-C', 'AC0345-C', 'AC0346-C', 'AC0347-C', 'AC0348-C', 'AC0349-C']
data accessed forK6andX
genotype_array_made
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

data_rechunked
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0

allele counts computed
30  0  0  0
30  0  0  0
30  0  0  0
30  

vcf file written
file closed


In [ ]:
#use bcftools concat to concatenate chromosomes
#bcftools concat -o file_concat.vcf.gz -Oz file_2R.vcf file_3R.vcf file_2L.vcf file_3L.vcf
#ignore X for now because with male sibs can't reconstruct genotype
#tabix -p vcf file.vcf.gz
#continue to use the vcf files with ASERead_counter_star as "vcfjoind"